***데이터 가져오기***

In [4]:
!conda install pandas --yes

Solving environment: done

## Package Plan ##

  environment location: /Users/wonbinchoi/opt/anaconda3/envs/new_project

  added / updated specs:
    - pandas


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    blas-1.0                   |         openblas          45 KB
    bottleneck-1.3.5           |   py38h67323c0_0         113 KB
    libopenblas-0.3.21         |       h54e7dc3_0         4.9 MB
    numexpr-2.8.4              |   py38h57a7bef_0         129 KB
    numpy-1.23.5               |   py38h57a7bef_0          11 KB
    numpy-base-1.23.5          |   py38hc93c6d9_0         6.4 MB
    pandas-1.5.3               |   py38h07fba90_0        11.3 MB
    ------------------------------------------------------------
                                           Total:        22.9 MB

The following NEW packages will be INSTALLED:

  blas               pkgs/main/osx-64::blas-1.0-openblas 
  bo

In [5]:
import pandas as pd
it_general = pd.read_csv('/Users/wonbinchoi/AutoNewsDB/뉴스.csv')
print(it_general.shape)
it_general.head()

(17494, 9)


,news_id,title,writer,content,writed_at,url,main_category,sub_category,platform
0,346630,"정부, '우주항공청' 연내 개청…백지신탁 예외 등 특혜 쏟는다",NaN,\n [서울=뉴시스] 정부가 올해 중 우리나라의 우주항공 분야 연구개발(R&D)을 ...,2023-03-02 00:00:00,NaN,IT/과학,IT 일반,네이버
1,346633,우주항공청 전문가 영입 `연봉 10억·외국인 허용`,이준기,\n '한국형 NASA(미 항공우주국)'로 출범하는 우주항공청이 최고의 민간 전문가...,2023-03-02 00:00:00,NaN,IT/과학,IT 일반,네이버
2,347804,"박정호의 투자유치 '매직'…""SK쉴더스 IPO보다 더 이득""",선한결,"\n ""SK쉴더스의 기업공개(IPO)를 철회해서 아쉬웠는데, IPO보다 더 높은 가...",2023-03-01 18:03:05,NaN,IT/과학,IT 일반,네이버
3,347806,글·그림·로고 제작·번역...인공지능에 다 맡겨 보니,NaN,\n 최근 텍스트 생성 인공지능(AI) 챗봇 챗GPT(ChatGPT)를 이용해 만든...,2023-03-01 18:02:03,NaN,IT/과학,IT 일반,네이버
4,347807,"LG엔솔-혼다, 美 배터리 공장 '첫삽'…기공식 개최",이한얼,\n LG에너지솔루션은 28일(현지시간) 오하이오 주 파이에트 카운티 제퍼슨빌 인근...,2023-03-01 18:26:01,NaN,IT/과학,IT 일반,네이버


# 클러스터링으로 문장 요약

***토큰화***

In [ ]:
# 문장 토큰화
text = docs[0]
sent_list = text.split('.')
sentences = [s.strip() for s in sent_list]
sentences
   

***mecab으로 전처리***

In [ ]:
# 현재 전처리 코드는 gensim에서는 사용하지 않고 나중에 사용할 예정 

In [ ]:
from konlpy.tag import Mecab

# 토큰화 => pos tagging => 품사 중 N과 V만 가져오는 함수. 
def mecab_preprocessor(text):
    mecab = Mecab()
    tokens = mecab.morphs(text)
    
    posed_list = []
    for token in tokens:
        posed_list += mecab.pos(token)
        
    return " ".join([token[0] for token in posed_list if token[1][0] in ['N', 'V']])
    
def return_preprocessed_docs(docs):
    preprocessed_docs = []
    for doc in docs:
        preprocessed_docs.append(mecab_preprocessor(doc))
    return preprocessed_docs

preprocessed_docs = return_preprocessed_docs(summed_docs)
preprocessed_docs

***feature vectorization***

In [ ]:
def sort_keywords(keywords):
    return sorted(zip(keywords.col, keywords.data), key=lambda x: (x[1], x[0]), reverse=True)
 
def extract_keywords(feature_names, sorted_keywords, n=5):
    return [(feature_names[idx], score) for idx, score in sorted_keywords[:n]]

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

stop_words = [
    '대표', '야구', '부회장', '자신', '대비'
]

tfidf_vect = TfidfVectorizer(stop_words=stop_words, max_df=.95, min_df=2, max_features=10000)
ftr_vect = tfidf_vect.fit_transform(preprocessed_docs)
sorted_keywords = sort_keywords(ftr_vect.tocoo())
feature_names = tfidf_vect.get_feature_names_out()

keywords = extract_keywords(feature_names, sorted_keywords, 15)
keywords_list = [keyword_tuple[0] for keyword_tuple in keywords]
keywords_list

In [ ]:
def remove_punct(text):
    """
    지우지 않는 punctuation 빼고 나머지 삭제 
    """
    
    # string의 translate() 메소드를 사용하여 punctuation 제거
    remove_punct_dict = dict((ord(i), " ") for i in string.punctuation \
                             if i not in ["."])
    
    return text.translate(remove_punct_dict)

temp_content = temp['content'].apply(lambda x : remove_punct(x))

***클렌징 코드***

In [ ]:
import re
import gc
import warnings
warnings.filterwarnings('ignore')

def clean_up_contents(text):
    """not preprocessing"""
    
    text = re.sub('[▶△▶️◀️▲▷ⓒ■◆●©️]', '', str(text)) # remove unnecessary markers
    text = re.sub('(\[?\[\w+)\s?\w{2,}?\s?(\w+\]\]?)\s+?(\[?\w{2,}?\]?)', '', text) # remove_reporter_01, [KBS 창원] [앵커], [이코노미스트 정두용 기자] 온라인 
    text = re.sub('(\[\w+\s+\w+.\s+\w+\s+\w+\s+\w+\s+\w+])', "", text) # '[산업통상자원부 제공. 재판매 및 DB 금지]'
    text = re.sub('(\(\w+)=(\w+\))(\s+\w+\s+\w+\s+)=', '', text) # remove_reporter_02
    text = re.sub('(\【)\w+\(\w+\)=\w+\s+\w+(\】)', '', text) # remove_reporter_03
    text = re.sub('(\[\w+\s?)(\(\w+\))?=(\s?\w+\s?\w+?\s?\w+?\])(\w{2,}?\s+?\w{2,}?\s+?)=?', ' ', text) # remove_reporter_04 revised, [서울=뉴시스]최은수 기자 = 
    #text = re.sub('(\[\w+\s?)=?(\s?\w+\])', ' ', text) # remove_reporter_04
    text = re.sub("(\[\w+\s+\w+in\s+\w+\s+\w+\])", "", text)# [이코노미스트 마켓in 허지은 기자] 
    text = re.sub('(\w+\(\w+\))=', ' ', text)
    text = re.sub('(\<)\w+\s*\d-?\d?(\>)', "", text)
    text = re.sub('(\<\w+\s+?\w+?\s+?):?(\w+\s+?\w+?\s+?\w+?\>)', "", text) # <삼성전자 OLED TV 라인업 현황> 
    text = re.sub('(\[\w+\s+\w+):(\s+\w+)-(\w+\]).*', '', text)
    text = re.sub('(\w+\(?=\w+\s?\w{2,}?\(?)', "", text) # 바르셀로나=선한결 기자, 바르셀로나(=스페인), 바르셀로나(=스페인)= 
    text = re.sub('(\w+\s+\w+\s+)기자', "", text) # 이혁기 더스쿠프 기자
    text = re.sub('KBS(\s+\w+\s+\w+).\s+(\w+:\w+\/\w+:\w+)', "", text) # KBS 뉴스 천현수입니다. 촬영기자:지승환/그래픽:박재희
    text = re.sub('(\[\w+\],\[\w+\])', "", text) # [뉴스리뷰],[앵커]
    text = re.sub('(연합뉴스TV\s+\w+)', "", text) # 연합뉴스TV 김장현입니다.
    text = re.sub('(\w+\s+기자)', "", text) # 손봉석 기자
    
    
    text = re.sub('(\w+)@(\w+).(\w+).(\w+)', ' ', text) # remove emails
    text = re.sub("(트위터\s+)(@)(\w+)", " ", text) # remove twitter accounts
#     text = re.sub("(\w+).(\w+)/(\w+)", " ", text) # remove facebook accounts
    text = text.replace('페이스북 /LeYN1', '')
    
    text = re.sub('(더 많은 글로벌투자).*', ' ', text) # remove uncessary words
    text = re.sub("(#[\w+]).*", "", text)
    text = text.replace(".,", ". ")
    
    
    text = re.sub('[ ]{2,}', ' ', text) # remove redundant empty spaces
    text = text.strip() # strip empty spaces
    if text[0] == ',':
        text = text.strip(',')
        
        
    return text


# 빈 콘텐츠 삭제 함수
def remove_empty_contents(df):
    
    for idx, content in enumerate(df['content_cleaned']):
        if content == '':
            df.drop(index=[idx], axis=0, inplace=True) 
        if content.count('.') < 2: # '.'이 없는 경우 관련없는 콘텐츠가 있었음. 
            df.drop(index=[idx], axis=0, inplace=True)
            
    df = df.reset_index(drop=True)
    return df

# 글자수가 일정 숫자 아래면 제거 -> why? 사진이 메인이고 사진을 설명하는 description일 가능성이 높음. 
def remove_contents_less_than_280(df):
    for ind, content in enumerate(df['content_cleaned']):
        if len(content) < 280:
            df.drop(index=[ind], axis=0, inplace=True)
    df = df.reset_index(drop=True)
    return df


# 최종 클렌징 코드
def cleanse_contents(df):
    
    df['content_cleaned'] = df['content'].apply(lambda x : clean_up_contents(x)) # content 클렌징 함수
    df = remove_empty_contents(df) # 컨텐츠가 빈 row 제거 
    df = remove_contents_less_than_280(df)
    
    gc.collect()
    
    return df


it_general = cleanse_contents(it_general)


***KMeans로 클러스터링 summarize 하기***

***요약된 문장 가져오기***

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from konlpy.tag import Mecab
from gensim.summarization import summarize
import numpy as np
import gc
import warnings
warnings.filterwarnings('ignore')

# tokenizer
def mecab_tokenizer(doc):
    mecab = Mecab()
    sentences = [sentences.strip() for sentences in doc.split('.')]
    pos_list = []
    for sent in sentences:
        for pos in mecab.pos(sent):
            if pos[1][0] in ['N']:
                if len(pos[0]) > 1:
                    pos_list.append(pos[0])

    return pos_list



def create_summarized_sentences(docs_df, n_clusters, n_top_words=7):
    
    docs = docs_df['content_cleaned']
    

    stop_words_list = ['대표이사', '대표', '이벤트', '컴투스', '넷마블', '프라시아 전기', '렐름',
        '사진공동취재단', 
        '산업통상자원부 제공',]

    tfidf_vect = TfidfVectorizer(
                                 tokenizer=mecab_tokenizer,
                                 max_df=.95,
                                 min_df=2, 
                                 ngram_range=(1,3), 
                                 stop_words=stop_words_list
                                 )
    ftr_vect = tfidf_vect.fit_transform(docs)
    

    # KMeans 사용하여 클러스터링 (문서 군집화)
    km_cluster = KMeans(n_clusters=3, max_iter=1000, random_state=0)
    km_cluster.fit(ftr_vect)
    
    # 문서를 label로 나누기 위해 km_cluster.labels_를 데이터프레임에 컬럼으로 넣어주기 
    docs_df['label'] = km_cluster.labels_
    
    # 요약된 문장 한 리스트 안에 넣어주기 
    summed_list = []
    for n in range(n_clusters):
        summed_list.append(" ".join(summarize("\n\n".join(docs_df[docs_df['label'] == n]['content_cleaned'].values.tolist()), 
                              ratio=.3, word_count=50, split=True)))
        
    for idx, results in enumerate(summed_list):
        print ("\n{} 번째 군집화의 요약된 문장 ==> {}".format(idx, results))
    
    gc.collect()
    
    return summed_list

it_general_summed_list = create_summarized_sentences(it_general, n_clusters=3, n_top_words=5) 
it_general_summed_list



# 토픽모델링으로 문장 요약

***하이퍼 파라미터 튜닝 + 문서 요약***

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer

"""토큰화 함수"""
def mecab_tokenizer(doc):
    mecab = Mecab()
    sentences = [sentences.strip() for sentences in doc.split('.')]
    pos_list = []
    for sent in sentences:
        for pos in mecab.pos(sent):
            if pos[1][0] in ['N']: # 우선 명사만 토큰화 
                if len(pos[0]) > 1:
                    pos_list.append(pos[0])

    return pos_list

"""하이퍼 파라미터 튜닝 함수"""
def hyperparms_tuning(train_data, n_jobs=1):
    lda_pipeline = Pipeline([               # tokenizer 바꿔서 써보기 
        ('tfidf_vect', TfidfVectorizer(tokenizer=mecab_tokenizer, max_df=.95, min_df=2, stop_words=['휘센', '대표'])),
        ('lda', LatentDirichletAllocation(max_iter=10, random_state=0, learning_method='online'))
    ])
    
    search_params = {
        'tfidf_vect__ngram_range' : [(1,1), (1,2), (1,3)],
        'tfidf_vect__use_idf' : (True, False),
        'lda__n_components' : [3,4,5]
    }
    
    gs_lda = GridSearchCV(lda_pipeline, param_grid=search_params, n_jobs=n_jobs)
    gs_lda = gs_lda.fit(train_data)
#     print('best score : {}'.format(gs_lda.best_score_))
    best_parameters = gs_lda.best_estimator_.get_params()
#     for params_name in sorted(list(best_parameters.keys())):
#         print('\t{0}: {1}'.format(params_name, best_parameters[params_name]))
    return gs_lda.best_estimator_


"""문장 요약 함수"""
def extract_summarized_sentences(df, column_name='content'):
    lda_fitted = hyperparms_tuning(df[column_name][:100]) # 여기서 [:100]는 나중에 빼주기 
    new_topics = lda_fitted.transform(df[column_name][:100])
    topic_names = ['Topic #' + str(i) for i in range(lda_fitted.get_params()['lda__n_components'])] # n_components 수만큼 Topic 가져오기 
    topic_df = pd.DataFrame(data=news_topics, index=df.title[:100], columns = topic_names)
    temp_df = topic_df.join(df[:100][[column_name, 'title']].set_index('title'), on='title')


    # 임계값 기준으로 토픽 나누기 
    threshold=0.5 # 임계값은 임의로 .5로 먼저 설정 
    topic_classfication_dict = dict()

    for topic in temp_df.columns[:-1].tolist(): # 마지막 컬럼 빼고 
        idx_list = [idx for idx, value in enumerate(temp_df[topic].values) if value > .5]
        value_list = [value for idx, value in enumerate(temp_df[topic].values) if value > .5]
        topic_classfication_dict[topic] = {}
        topic_classfication_dict[topic]['idx'] = idx_list
        topic_classfication_dict[topic]['value'] = value_list

    # 토픽별 인덱스 값으로 뉴스 요약 출력하기 
    for com_num in range(lda_fitted.get_params()['lda__n_components']):
        topic_idx = [idx for idx in topic_classfication_dict[f'Topic #{com_num}']['idx']]
        print ("-"*110)            
        print (summarize("".join(temp_df.iloc[topic_idx][column_name].tolist()), word_count=30, ratio=.2))
                                                                                        # 문장길이 조절 가능 
    return


extract_summarized_sentences(it_general, column_name = 'reduced_content')

# """마지막으로 요약된 문장 내 이메일로 보내는 함수"""




***문장 길이 조절 후 요약해보기***

In [ ]:
from gensim.summarization import summarize
import string
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

def remove_punct(text):
    remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation if punct not in ['.'])
    return text.translate(remove_punct_dict)
    
len_list = [len(content) for content in it_general['content_cleaned'].apply(lambda x : remove_punct(x))]

new_content_list = []
for content in it_general['content_cleaned']:
    if len(remove_punct(content)) > np.median(len_list):
        new_content_list.append(summarize(content, word_count=np.median(len_list)))
    else:
        new_content_list.append(content)

In [ ]:
it_general['reduced_content'] = new_content_list

In [ ]:
# 문장 길이 변화전 문장 길이 분포도 
len_list = [len(content) for content in it_general['content_cleaned']]
plt.hist(len_list, bins=50)
plt.show()

In [ ]:
# 문장 길이 변화 후 분포도 
new_len_list = [len(content) for content in new_content_list]

plt.hist(new_len_list, bins=50)
plt.show()

In [ ]:
"""문장 길이 변화후 결론 ==> 요약에 크게 변화된 점이 발견되지 않음. """

***문장별 전처리 코드 (여기서는 사용하지 않음)***

In [ ]:
import string
def remove_punct(sent):
    remove_punct_dict = dict((ord(punct), " ") for punct in string.punctuation if punct not in ['(', ')', ',', '&', '"'])
    return sent.translate(remove_punct_dict)

doc = it_general['content_cleaned'][2]
sentence = ''
for sent in doc.split('.'):
    sent = sent.strip()
    sent = remove_punct(sent)
    if len(sent) == 0:
        continue
    sent = sent.strip()
    sent += '. '
    sentence += sent
    
re.sub('[ ]{2,}',' ',sentence)

***잠깐 전처리***

In [ ]:
# [ 김현일·] 없애기 

In [6]:
!conda install -c conda-forge scikit-learn --yes

Solving environment: done

## Package Plan ##

  environment location: /Users/wonbinchoi/opt/anaconda3/envs/new_project

  added / updated specs:
    - scikit-learn


The following NEW packages will be INSTALLED:

  joblib             conda-forge/noarch::joblib-1.2.0-pyhd8ed1ab_0 
  libblas            conda-forge/osx-64::libblas-3.9.0-16_osx64_openblas 
  libcblas           conda-forge/osx-64::libcblas-3.9.0-16_osx64_openblas 
  liblapack          conda-forge/osx-64::liblapack-3.9.0-16_osx64_openblas 
  pooch              conda-forge/noarch::pooch-1.7.0-pyha770c72_3 
  python_abi         conda-forge/osx-64::python_abi-3.8-2_cp38 
  scikit-learn       conda-forge/osx-64::scikit-learn-1.2.2-py38h1081964_1 
  scipy              conda-forge/osx-64::scipy-1.10.1-py38hfb8b963_0 
  threadpoolctl      conda-forge/noarch::threadpoolctl-3.1.0-pyh8a188c0_0 

The following packages will be SUPERSEDED by a higher-priority channel:

  ca-certificates    pkgs/main::ca-certificates-2023.01.10~ --> con

In [8]:
!conda install gensim==3.8.3 --yes

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: done

## Package Plan ##

  environment location: /Users/wonbinchoi/opt/anaconda3/envs/new_project

  added / updated specs:
    - gensim==3.8.3


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    gensim-3.8.3               |   py38h23ab428_2        18.4 MB
    smart_open-5.2.1           |   py38hecd8cb5_0          77 KB
    ------------------------------------------------------------
                                           Total:        18.5 MB

The following NEW packages will be INSTALLED:

  gensim             pkgs/main/osx-64::gensim-3.8.3-py38h23ab428_2 
  smart_open         pkgs/main/osx-64::smart_open-5.2.1-py38hecd8cb5_0 

The following packages will be UPDATED:

  ca-certificates    conda-forge::ca-certificates-2022.12.~ --> pkgs/main::ca-certificates-2023.01.10-hecd8cb5_0 

In [10]:
from konlpy.tag import Mecab
mecab = Mecab()
from gensim.summarization import summarize

**********************************************

In [42]:
from os.path import basename
import numpy as np
import pandas as pd
from datetime import datetime
import csv

# from scrapy.crawler import CrawlerProcess
# from scrapy.utils.project import get_project_settings

# from nltk.tokenize import sent_tokenize
from konlpy.tag import Mecab
from gensim.summarization import summarize # keywords

from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

# import smtplib
# import email
# from email.mime.multipart import MIMEMultipart
# from email.mime.text import MIMEText
# from email.mime.base import MIMEBase
# from email import encoders
# from email.header import Header
# from bs4 import BeautifulSoup

In [12]:
from datetime import datetime


# scrapy 변수 설정, settings에서 데이터 이름, 형식 지정
SPIDER_NAME = 'naver_spider'
PATH = 'C:/Users/tlrks/Desktop/workspace/exercise/NLP/ScrapyNewsCrawler/Naver_news/Naver_news/test.csv'

# email 변수 설정
SMTP_SERVER = ''
SMTP_PORT = 465
SMTP_USER = ''
SMTP_PASSWORD = ''

to_users = ['prbl@kakao.com']
target_addr = ','.join(to_users)
subject = datetime.now().date().strftime('%Y. %m. %d') + '기사 요약 메일'

# 불용어 불러오기
stopword_file = open('/Users/wonbinchoi/Downloads/stopword.txt', 'r', encoding='utf-8')
stopwords= []
for word in stopword_file.readlines():
    stopwords.append(word.rstrip())
stopword_file.close()

CATEGORIES = {
        ('경제', 101) : {
            '금융' : '259',
            '증권' : '258',
            '산업/재계' : '261',
            '중기/벤처' : '771',
            '부동산' : '260',
            '글로벌 경제' : '262',
            '생활경제' : '310',
            '경제 일반' : '263'
        },
        ('IT/과학', 105) : {
            '모바일' : '731',
            '인터넷/SNS' : '226',
            '통신/뉴미디어' : '227',
            'IT 일반' : '230',
            '보안/해킹' : '732',
            '컴퓨터' : '283',
            '게임/리뷰' : '229',
            '과학 일반' : '228'
        }
    }

In [39]:
data = pd.read_csv('/Users/wonbinchoi/Downloads/day_data.csv', encoding='utf-8')
data.shape

(6436, 10)

In [40]:


# scrapy crawler 실행하는 함수(파일 경로는 settings에서 설정)
# def run_spider():
#     process = CrawlerProcess(get_project_settings())
#     process.crawl(SPIDER_NAME)
#     process.start()

# # 크롤링한 데이터 불러오는 함수
# def load_data(path):
#     data = pd.read_csv(path, encoding='utf-8', dtype=object)
#     # print(data.groupby(by=['main_category', 'sub_category']).id.count())
#     return data

# 사망기사 관련 키워드 두 개 이상 포함 시 삭제
def isObituary(text):
    words = ['별세', '장례', '발인', '부고', '친상']
    count = 0
    for word in words:
        if word in text:
            count += 1
            if count > 1:
                return True
    return False

# 주어진 길이보다 긴 경우 True 반환
def isLonger(text, length):
    if len(text) > length:
        return True
    else:
        return False

# 데이터를 전처리하는 함수
def preprocess(data):
    print('처리 전 기사 수 :', len(data))
    data.dropna(subset=['content'], inplace=True)
    data.drop_duplicates(subset=['title', 'writer'], keep='first', inplace=True)
    data.drop_duplicates(subset=['title', 'content'], keep='first', inplace=True)
    data = data[data.content.apply(isLonger, args=[150])]
    data = data[~data.content.apply(isObituary)]
    data = data[~data.title.str.contains('\[인사\]')]
    data = data[~data.title.str.contains('\[.*부고.*\]')]
    data = data[~data.title.str.contains('\[포토뉴스\]')]
    data = data[~data.title.str.contains('코스피') | data.content.apply(isLonger, args=[300])]
    data = data[~data.title.str.contains('증시') | data.content.apply(isLonger, args=[300])]
    print('처리 후 기사 수 :', len(data))
    data['tokens'] = data.content.apply(mecab_tokenizer)
    return data

def mecab_tokenizer(doc):
#     mecab = Mecab('C:/mecab/mecab-ko-dic')
    sentences = [sentences.strip() for sentences in doc.split('.')]
    pos_list = []
    for sent in sentences:
        for pos in mecab.pos(sent):
            if pos[1][0] in ['N']: # 우선 명사만 토큰화 
                if len(pos[0]) > 1:
                    if pos[0] not in stopwords:
                        pos_list.append(pos[0])
    return ' '.join(pos_list)

def find_best_lda(train_data, n_jobs=-1):
    lda_pipeline = Pipeline([
        ('tfidf_vect', TfidfVectorizer(max_df=.9, min_df=2, lowercase=False, use_idf=False)),
        ('lda', LatentDirichletAllocation(max_iter=10, random_state=0, learning_method='online'))
    ])
    
    search_params = {
        'tfidf_vect__max_features' : [500, 700, 1000],
        'tfidf_vect__ngram_range' : [(1,1), (1,2)],
        'lda__n_components' : [3, 5, 10]
    }
    
    gs_lda = GridSearchCV(lda_pipeline, param_grid=search_params, n_jobs=n_jobs)
    gs_lda = gs_lda.fit(train_data)
    # print('best score : {}'.format(gs_lda.best_score_))
    # best_parameters = gs_lda.best_estimator_.get_params()
    # for params_name in sorted(list(best_parameters.keys())):
    #     print('\t{0}: {1}'.format(params_name, best_parameters[params_name]))
    return gs_lda.best_estimator_

# lda의 토픽별 문서 인덱스가 담긴 리스트 반환
def get_idxs_by_topic(model, train_data, threshold=0.5):
    doc_topic_matrix = model.transform(train_data)
    idxs_by_topic = []
    for i in range(model.get_params()['lda__n_components']):
        idxs = []
        for j, doc in enumerate(doc_topic_matrix):
            max_topic = np.argmax(doc)
            max_prob = doc[max_topic]
            if max_topic == i and max_prob >= threshold:
                idxs.append(j)
        idxs_by_topic.append(idxs)
    return idxs_by_topic

# 서브카테고리별 가장 많이 나타난 토픽에 대한 상위기사 k개를 찾아 딕셔너리 형태로 반환
def top_articles_by_category(preprocessed_data, k=1):
    print(datetime.now().strftime('%H:%M:%S')+' 카테고리별 기사 추출 시작')
    SUB_CATEGORIES = [] # 서브카테고리 목록
    top_articles = {}   # 서브카테고리별 대표기사의 제목과 url
    for main_category in CATEGORIES:
        for sub_category in CATEGORIES[main_category].items():
            SUB_CATEGORIES.append(sub_category[0])
            top_articles.setdefault(sub_category[0], [])

    for sub_category in SUB_CATEGORIES:
        category_data = preprocessed_data[preprocessed_data.sub_category==sub_category]
        category_content = category_data.content.values
        category_tokens = category_data.tokens.values
        # 파라미터 튜닝, 토픽 개수 저장
        best_model = find_best_lda(category_tokens)
        number_of_topic = best_model.get_params()['lda__n_components']

        # 토픽별 인덱스 리스트를 가져와 가장 많이 나타난 토픽의 인덱스 찾기
        idxs_by_topic = get_idxs_by_topic(best_model, category_tokens, 0.6)
        topic_count = list(map(len, idxs_by_topic))
        most_topic = np.argmax(topic_count)

        # lda 점수가 가장 높은 문서 찾아 제목과 url 저장
        most_topic_weights = best_model.transform(category_tokens)[:, most_topic]
        top_doc_idx = np.argsort(most_topic_weights)[::-1][:k]
        for idx in top_doc_idx:
            top_articles[sub_category].append((
                        category_data.title.values[idx],
                        category_data.link.values[idx],
                        summarize(category_content[idx], word_count=20, ratio=.2)
        ))
        print(datetime.now().strftime('%H:%M:%S')+sub_category+' 추출 완료')
    print(top_articles)
    return top_articles

def main():
    preprocessed_data = preprocess(data)

    # 각 서브카테고리별로 상위기사 1개에 대해 제목, 링크, 요약을 top_article에 저장
    top_article = top_articles_by_category(preprocessed_data, 1)
    print (top_article)
    return 

if __name__=='__main__':
    main()

처리 전 기사 수 : 6436
처리 후 기사 수 : 5989
11:59:06 카테고리별 기사 추출 시작
11:59:28금융 추출 완료
11:59:53증권 추출 완료
12:00:46산업/재계 추출 완료
12:00:54중기/벤처 추출 완료
12:01:10부동산 추출 완료
12:01:18글로벌 경제 추출 완료
12:01:37생활경제 추출 완료
12:02:48경제 일반 추출 완료
12:02:50모바일 추출 완료
12:02:55인터넷/SNS 추출 완료
12:03:00통신/뉴미디어 추출 완료
12:03:21IT 일반 추출 완료
12:03:22보안/해킹 추출 완료
12:03:25컴퓨터 추출 완료
12:03:29게임/리뷰 추출 완료
12:03:34과학 일반 추출 완료
{'금융': [('추경호, IDB 총재 만나…“韓 중남미 진출 교두보 마련 협조 당부”', 'https://n.news.naver.com/mnews/article/030/0003090951?sid=101', '추경호 부총리 겸 기획재정부 장관(오른쪽)이 12일(현지시간) 미국 국제통화기금(IMF)에서 국제신용평가사 피치(Fitch)의 제임스 맥코맥 국가신용등급 글로벌총괄과 악수하며 인사를 나누고 있다.\n추경호 부총리 겸 기획재정부 장관(오른쪽)이 12일(현지시간) 국제통화기금(IMF)에서 마그달레나 제치코브스카 폴란드 재무장관과 면담에 앞서 악수하며 기념사진을 찍고 있다.')], '증권': [('물가 호재보다 경기침체가…"어닝 쇼크 주의, 변동성 대비해야"', 'https://n.news.naver.com/mnews/article/008/0004874791?sid=101', '임종철 디자인기자 /사진=임종철 디자인기자 경기침체 우려가 커지며 증시의 상승 동력도 점차 약해진다.\n인플레이션 완화 기대감에 미국 증시는 장 초반 상승세를 보였으나 곧 경기침체 우려가 부각되며 하락 전환했다.')], '산업/재계': [('S-OIL, 전기차 전용 윤활유 브랜드 ‘세븐 EV’ 출시', 'https://n.news.nav

In [55]:


# scrapy crawler 실행하는 함수(파일 경로는 settings에서 설정)
# def run_spider():
#     process = CrawlerProcess(get_project_settings())
#     process.crawl(SPIDER_NAME)
#     process.start()

# # 크롤링한 데이터 불러오는 함수
# def load_data(path):
#     data = pd.read_csv(path, encoding='utf-8', dtype=object)
#     # print(data.groupby(by=['main_category', 'sub_category']).id.count())
#     return data

# 사망기사 관련 키워드 두 개 이상 포함 시 삭제

def save_articles(top_articles):
    fname = ['category', 'title', 'link', 'summary']
    with open('./articles.csv', 'w') as f:
        w = csv.DictWriter(f, fieldnames=fname)
        w.writeheader()
        w.writerows(top_articles)
        return
    
    
def isObituary(text):
    words = ['별세', '장례', '발인', '부고', '친상']
    count = 0
    for word in words:
        if word in text:
            count += 1
            if count > 1:
                return True
    return False

# 주어진 길이보다 긴 경우 True 반환
def isLonger(text, length):
    if len(text) > length:
        return True
    else:
        return False

# 데이터를 전처리하는 함수
def preprocess(data):
    print('처리 전 기사 수 :', len(data))
    data.dropna(subset=['content'], inplace=True)
    data.drop_duplicates(subset=['title', 'writer'], keep='first', inplace=True)
    data.drop_duplicates(subset=['title', 'content'], keep='first', inplace=True)
    data = data[data.content.apply(isLonger, args=[150])]
    data = data[~data.content.apply(isObituary)]
    data = data[~data.title.str.contains('\[인사\]')]
    data = data[~data.title.str.contains('\[.*부고.*\]')]
    data = data[~data.title.str.contains('\[포토뉴스\]')]
    data = data[~data.title.str.contains('코스피') | data.content.apply(isLonger, args=[300])]
    data = data[~data.title.str.contains('증시') | data.content.apply(isLonger, args=[300])]
    print('처리 후 기사 수 :', len(data))
    data['tokens'] = data.content.apply(mecab_tokenizer)
    return data

def mecab_tokenizer(doc):
#     mecab = Mecab('C:/mecab/mecab-ko-dic')
    sentences = [sentences.strip() for sentences in doc.split('.')]
    pos_list = []
    for sent in sentences:
        for pos in mecab.pos(sent):
            if pos[1][0] in ['N']: # 우선 명사만 토큰화 
                if len(pos[0]) > 1:
                    if pos[0] not in stopwords:
                        pos_list.append(pos[0])
    return ' '.join(pos_list)

def find_best_lda(train_data, n_jobs=-1):
    lda_pipeline = Pipeline([
        ('tfidf_vect', TfidfVectorizer(max_df=.9, min_df=2, lowercase=False, use_idf=False)),
        ('lda', LatentDirichletAllocation(max_iter=10, random_state=0, learning_method='online'))
    ])
    
    search_params = {
        'tfidf_vect__max_features' : [500, 700, 1000],
        'tfidf_vect__ngram_range' : [(1,1), (1,2)],
        'lda__n_components' : [3, 5, 10]
    }
    
    gs_lda = GridSearchCV(lda_pipeline, param_grid=search_params, n_jobs=n_jobs)
    gs_lda = gs_lda.fit(train_data)
    # print('best score : {}'.format(gs_lda.best_score_))
    # best_parameters = gs_lda.best_estimator_.get_params()
    # for params_name in sorted(list(best_parameters.keys())):
    #     print('\t{0}: {1}'.format(params_name, best_parameters[params_name]))
    return gs_lda.best_estimator_

# lda의 토픽별 문서 인덱스가 담긴 리스트 반환
def get_idxs_by_topic(model, train_data, threshold=0.5):
    doc_topic_matrix = model.transform(train_data)
    idxs_by_topic = []
    for i in range(model.get_params()['lda__n_components']):
        idxs = []
        for j, doc in enumerate(doc_topic_matrix):
            max_topic = np.argmax(doc)
            max_prob = doc[max_topic]
            if max_topic == i and max_prob >= threshold:
                idxs.append(j)
        idxs_by_topic.append(idxs)
    return idxs_by_topic

# 서브카테고리별 가장 많이 나타난 토픽에 대한 상위기사 k개를 찾아 딕셔너리 형태로 반환
def top_articles_by_category(preprocessed_data, k=1):
    print(datetime.now().strftime('%H:%M:%S')+' 카테고리별 기사 추출 시작')
    SUB_CATEGORIES = [] # 서브카테고리 목록
    top_articles = []   # 서브카테고리별 대표기사의 제목과 url
    for main_category in CATEGORIES:
        for sub_category in CATEGORIES[main_category].items():
            SUB_CATEGORIES.append(sub_category[0])
            # top_articles.setdefault(sub_category[0], [])

    for sub_category in SUB_CATEGORIES:
        category_data = preprocessed_data[preprocessed_data.sub_category==sub_category]
        category_content = category_data.content.values
        category_tokens = category_data.tokens.values
        # 파라미터 튜닝, 토픽 개수 저장
        best_model = find_best_lda(category_tokens)
        number_of_topic = best_model.get_params()['lda__n_components']

        # 토픽별 인덱스 리스트를 가져와 가장 많이 나타난 토픽의 인덱스 찾기
        idxs_by_topic = get_idxs_by_topic(best_model, category_tokens, 0.6)
        topic_count = list(map(len, idxs_by_topic))
        most_topic = np.argmax(topic_count)

        # lda 점수가 가장 높은 문서 찾아 제목과 url 저장
        most_topic_weights = best_model.transform(category_tokens)[:, most_topic]
        top_doc_idx = np.argsort(most_topic_weights)[::-1][:k]
        for idx in top_doc_idx:
            top_articles.append({
                                'category' : sub_category,
                        'title' : category_data.title.values[idx],
                        'link' : category_data.link.values[idx],
                                'summary' : summarize(category_content[idx], word_count=20, ratio=.2)
            })
        print(datetime.now().strftime('%H:%M:%S'), sub_category, '추출 완료')

    return top_articles

def main():
    preprocessed_data = preprocess(data)

    # 각 서브카테고리별로 상위기사 1개에 대해 제목, 링크, 요약을 top_article에 저장
    top_article = top_articles_by_category(preprocessed_data, 1)
    save_articles(top_article)
    return 

if __name__=='__main__':
    main()



처리 전 기사 수 : 6436
처리 후 기사 수 : 5989
13:36:40 카테고리별 기사 추출 시작
13:37:01 금융 추출 완료
13:37:26 증권 추출 완료
13:38:18 산업/재계 추출 완료
13:38:27 중기/벤처 추출 완료
13:38:45 부동산 추출 완료
13:38:52 글로벌 경제 추출 완료
13:39:15 생활경제 추출 완료
13:40:32 경제 일반 추출 완료
13:40:35 모바일 추출 완료
13:40:39 인터넷/SNS 추출 완료
13:40:45 통신/뉴미디어 추출 완료
13:41:08 IT 일반 추출 완료
13:41:09 보안/해킹 추출 완료
13:41:12 컴퓨터 추출 완료
13:41:16 게임/리뷰 추출 완료
13:41:21 과학 일반 추출 완료


0406_word2vec_구현.ipynb
README.md
Untitled.ipynb
articles.csv
k_means_practice.ipynb
kdt_lesson_learned_pca.ipynb
kdt_ll_Apr_4th_RPA.ipynb
kdt_nlp_project_notebook_and_lesson_learned.ipynb
lesson_learned_kdt_NLP_0320.ipynb
lesson_learned_kdt_단어의_표현_0321.ipynb
lesson_learned_kdt_알고리즘&자료구조_1일차_and_2일차.ipynb
lesson_learned_kdt_알고리즘&자료구조_3일차.ipynb
news_4_project.ipynb


In [57]:
temp = pd.read_csv('./articles.csv')
print(temp['summary'][0])

추경호 부총리 겸 기획재정부 장관(오른쪽)이 12일(현지시간) 미국 국제통화기금(IMF)에서 국제신용평가사 피치(Fitch)의 제임스 맥코맥 국가신용등급 글로벌총괄과 악수하며 인사를 나누고 있다.
